# ST446 Distributed Computing for Big Data

## Week 4 class: Spark SQL and Hive integration

### LT 2020

In this exercise, we will connect Spark with a local Hive installation that uses mysql metastore. We will demonstrate that the same tables can be accessed and manipulated from both Hive and Spark. We can persist tables and query them using either Hive or Spark.

## 0. Preparation

Beforing running this jupyter notebook, make sure you completed all the commands in the previous three tutorials.

In [1]:
spark.sql("show databases").show() # your output may differ!

+------------+
|databaseName|
+------------+
|        dblp|
|     default|
|      fbflow|
+------------+



## 1. Create and remove tables using Spark SQL

#### Create

In [2]:
spark.sql("create table mytable (key string, val string)").collect()

[]

In [3]:
spark.sql("show tables").show() # your output may differ!

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  mytable|      false|
+--------+---------+-----------+



You can access the hive shell by opening a terminal on the VM and typing `hive`. Leave the hive shell via `ctrl-d`.
Check that you see the same from the Hive shell:
``` 
hive> use default;
OK
Time taken: 0.028 seconds
hive> show tables;
OK
mytable
word_count
Time taken: 0.031 seconds, Fetched: 2 row(s)
hive>
```

You should see mytable when you execute the command `show tables`.

In [4]:
spark.sql("describe mytable").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|     key|   string|   null|
|     val|   string|   null|
+--------+---------+-------+



#### Remove
Now, remove the table:

In [5]:
spark.sql("drop table mytable")

DataFrame[]

In [6]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



Check from the Hive shell that the table has been removed:
```
hive> show tables;
OK
Time taken: 0.604 seconds
hive>
```
You should see that mytable no longer exists.

## 2. Running a query script

Upload `week04/class/test-query.sql` and `week04/class/people.json` files into a folder under bucket, here called `gs://jialin-bucket/spark-hive/`.

Spark SQL SparkSession.sql() does not support running multiple query statements. A way around this is to write a script file and run each statement separately:

copy two files to your cluster
```
gsutil cp gs://jialin-bucket/spark-hive/test-query.sql .
```
and
```
gsutil cp gs://jialin-bucket/spark-hive/people.json .
```

In [8]:
with open("test-query.sql") as fr:
   query = fr.read()

for q in query.split(";")[:-1]:
    print(q) 
    result = spark.sql(q).show()


SHOW TABLES
+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+


DROP TABLE IF EXISTS mytable
++
||
++
++


CREATE TABLE IF NOT EXISTS mytable (key STRING, val STRING)
++
||
++
++


SHOW TABLES
+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  mytable|      false|
+--------+---------+-----------+


DESCRIBE mytable
+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|     key|   string|   null|
|     val|   string|   null|
+--------+---------+-------+


DROP TABLE mytable
++
||
++
++


SHOW TABLES
+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



## 3. Create and save a table using `spark.read.json` and `.write.saveAsTable`

In [9]:
df = spark.read.json('gs://jialin-bucket/spark-hive/people.json')

In [10]:
df.write.saveAsTable("people")

In [11]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|   people|      false|
+--------+---------+-----------+



In [12]:
spark.sql("describe people").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|     age|   bigint|   null|
|    name|   string|   null|
+--------+---------+-------+



In [13]:
# delete it again...
spark.sql("drop table people")

DataFrame[]

In [14]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

